In [1]:
import requests
import time
import random
import platform
import os
import json
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm_notebook as tqdm
from lxml import etree
from selenium import webdriver

In [13]:
class CrawlDoubanTV():
    
    user_agent = [
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60',
        'Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
        'Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)',
        'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0',
        'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)',
    ]
    
    def __init__(self, scrolls=25, headless=True, sleep=0.2, discover=None):
        self.scrolls = scrolls
        self.options = webdriver.ChromeOptions()
        self.ids = []
        self.sleep = sleep
        self.discover = discover
        self.result = {}
        if headless == True:
            self.options.add_argument('--headless')
            self.options.add_argument('--disable-gpu')
            
    def run(self):
        self._crawl_main()
        self._crawl_detail()
    
    def _crawl_main(self):
        if platform.system() == 'Windows':
            driver = webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe",
                                                  options=self.options)

        if platform.system() == 'Darwin':
            driver = webdriver.Chrome("/Users/Lunar/chromedriver/chromedriver",
                                                  options=self.options)
        
        print('Crawling main...')
        driver.get('https://movie.douban.com/tv/')
        time.sleep(1)
        
        driver.find_element_by_xpath("//input[@value='国产剧']/..").click()
        time.sleep(1)
        
        for i in range(self.scrolls):
            try:
                if driver.find_element_by_xpath("//a[text()='加载更多']").is_displayed():
                    driver.find_element_by_xpath("//a[text()='加载更多']").click()
                    time.sleep(1+random.random())
            except:
                time.sleep(3)
            print('{} / {}'.format(i+1, self.scrolls), end='\r')
            
        f = etree.HTML(driver.page_source)
        self.ids += f.xpath('//div[@class=\'cover-wp\']/@data-id')
            
        print(len(self.ids), self.ids[:5])
        print('Main crawled')
        
    def _generate_tv_pages(self, id_):
        head = 'https://movie.douban.com/subject/'
        tail = random.choice(['/?tag=国产剧&from=gaia_video', '/?from=subject-page'])
        return head + str(id_) + tail
    
    def _crawl_detail(self):
        print('Crawling details...')
        if self.discover:
            iters = self.discover
        else:
            iters = len(self.ids)
        for _ in tqdm(range(iters)):
            try:
                if len(self.ids) > 0:
                    id_ = random.choice(self.ids)
                    url = self._generate_tv_pages(id_)
                    headers = {'user-agent': random.choice(self.user_agent)}
                    text = requests.get(url=url, headers=headers).text
                    if self.discover:
                        self._discover_detail(text)
                    self._parse_details(text)
                    self.ids.remove(id_)
                    time.sleep(self.sleep + random.random() * self.sleep)
                else: break
            except: pass
        print('Done')
    
    # discvoer url from detail page and update the ids to crawl
    def _discover_detail(self, text):
        f = etree.HTML(text)
        found = f.xpath("//div[@id='recommendations']//dt/a[@data-moreurl-dict]/@data-moreurl-dict")
        found =  [eval(i)['subject_id'] for i in found]
        found = [i for i in found if i not in self.result]
        self.ids += found
        self.ids = list(set(self.ids))
        
    def _parse_details(self, text):
        f = etree.HTML(text)
        content = json.loads(f.xpath("//script[@type='application/ld+json']")[0].text)
        
        id_ = content['url'].split('/')[-2]
        parsed = {}
        parsed['name'] = content['name']
        parsed['date_published'] = content['datePublished']
        parsed['director'] = {i['url'].split('/')[-2]: i['name'].split(' ')[0] for i in content['director']}
        parsed['author'] = {i['url'].split('/')[-2]: i['name'].split(' ')[0] for i in content['author']}

        hold =[(i['url'].split('/')[-2], i['name'].split(' ')[0]) for i in content['actor']]
        parsed['main_actor'] = {id_: name for id_, name in hold[:5]}
        parsed['support_actor'] = {id_: name for id_, name in hold[5:]}

        parsed['genre'] = content['genre']
        parsed['duration'] = content['duration']
        parsed['rating'] = content['aggregateRating']['ratingValue']
        parsed['rating_count'] = content['aggregateRating']['ratingCount']
        parsed['rating_best'] = content['aggregateRating']['bestRating']
        parsed['rating_worst'] = content['aggregateRating']['worstRating']

        parsed['location'] = f.xpath("//div[@id='info']//span[text()='制片国家/地区:']")[0].tail.strip()
        parsed['num_episode'] = f.xpath("//div[@id='info']//span[text()='集数:']")[0].tail.strip()
                
        self.result[id_] = parsed



def clean(dic):
    id_to_pop = []
    for i in dic:
        if (len(dic[i]['author']) == 0 or len(dic[i]['director']) == 0) or ((
            len(dic[i]['main_actor']) == 0 or len(dic[i]['support_actor']) == 0) or 
            dic[i]['location'] != '中国大陆'):
            id_to_pop.append(i)
    for i in id_to_pop:
        dic.pop(i)
    return dic

def get_true_score(dic, typ='main_actor'):
    char_dic = get_mapping(dic)
    char_list = list(char_dic.keys())
    tv_list = list(dic.keys())
    rating_list = [float(dic[i]['rating']) for i in tv_list]
    df = pd.DataFrame(index = tv_list, columns=char_list).fillna(0)
    for i in tqdm(tv_list):
        for j in char_list:
            if j in dic[i][typ]:
                df.loc[i, j] = 1
    count = df.sum(1).values
    df = df / count[:, None]
    scores = np.linalg.pinv(df.values) @ np.array(rating_list).reshape(-1, 1)
    scores = {i: j for (i, j) in zip(char_list, scores)}
    count = {i: j for (i, j) in zip(char_list, count)}
    return scores, count

def get_avg_score(dic, typ='main_actor'):
    char_dic = get_mapping(dic)
    char_list = list(char_dic.keys())
    tv_list = list(dic.keys())
    rating_list = [float(dic[i]['rating']) for i in tv_list]
    df = pd.DataFrame(index = tv_list, columns=char_list)
    for i in tqdm(tv_list):
        for j in char_list:
            if j in dic[i][typ]:
                df.loc[i, j] = float(dic[i]['rating'])
    df['score'] = df.mean(1)
    return {i: j for (i, j) in zip(char_list, df['score'])}

def get_mapping(dic):
    char_dic= {}
    for i in dic:
        char_dic = dict(char_dic, **dic[i]['author'])
        char_dic = dict(char_dic, **dic[i]['director'])
        char_dic = dict(char_dic, **dic[i]['main_actor'])
        char_dic = dict(char_dic, **dic[i]['support_actor'])
    return char_dic

In [6]:
crawl = CrawlDoubanTV(scrolls=25, discover=1500, sleep=0.3)
crawl.run()

Crawling main...
400/ 25 ['27195020', '26794191', '26660368', '33413766', '30191989']
Main crawled
Crawling details...

Done


In [11]:
mapping = get_mapping(crawl.result)
print('Num of shows crawed: %s'%len(crawl.result))
print('Num of chars crawed: %s'%len(mapping))

Num of shows crawed: 355
Num of chars crawed: 5986


In [15]:
dic = crawl.result.copy()
dic = clean(dic)

main_avg = get_avg_score(clean(dic), typ='main_actor')
main_true, main_count = get_true_score(clean(dic), typ='main_actor')
support_avg = get_avg_score(clean(dic), typ='support_actor')
support_true, support_count = get_true_score(clean(dic), typ='support_actor')


df = pd.DataFrame(main_avg, index=['main_avg']).T.join(
       pd.DataFrame(main_true, index=['main_true']).T).join(
       pd.DataFrame(support_avg, index=['support_avg']).T).join(
       pd.DataFrame(support_true, index=['support_true']).T).join(
       pd.DataFrame(main_count, index=['main_count']).T).join(
       pd.DataFrame(support_count, index=['support_count']).T)

df['true_score'] = np.round(df['main_true'] * 0.7 + df['support_true'] * 0.3, 2)
df.loc[df['true_score'] > 10, 'true_score'] = 10
df.loc[df['true_score'] < 0, 'true_score'] = 0
df['true_score'] = np.abs(df['true_score'])
df['avg_score'] = np.round(df['main_avg'] * 0.7 + df['support_avg'] * 0.3, 2)
df['total'] = df['main_count'] + df['support_count']
df['main_pct'] = df['main_count'] / df['total']
df['diff'] = df['true_score'] - df['avg_score']
        
df = df.drop(['main_avg', 'main_true', 'support_avg', 'support_true', 'main_count', 'support_count'], axis=1)
df = df.rename(index=mapping)

In [17]:
df.to_excel('res.xlsx')